In [ ]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import bs4

In [ ]:
def get_page(url: str) -> bs4.BeautifulSoup:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    return soup

In [ ]:
SAVE_PATH = "/Users/yuvlo/Spot-the-bot/Russian/RuBooks/"

# aldebaran.ru

In [ ]:
for page_index in tqdm(range(250)):
    url = "https://aldebaran.ru/genre/klassika/russkaya_klassika/pagenum-" + str(page_index) + "/?sort=pop"
    soup = get_page(url)
    downloaded_htmls = soup.find_all('p', class_="booktitle")
    for html in downloaded_htmls:
        url = "https://aldebaran.ru" + html.find('a').get('href')
        soup = get_page(url)
        links_soup = soup.find('div', class_="item_download item_info border_bottom")
        if links_soup.text[:17] != "Скачать бесплатно":
            continue
        links_list_soup = links_soup.find_all('a')
        for link_soup in links_list_soup:
            if link_soup.text == "ePub":
                book_name = '_'.join(link_soup.get('href').split(sep='/')[2:4]) + ".epub"
                save_name = SAVE_PATH + book_name
                if os.path.isfile(save_name):
                    print('here', save_name)
                    continue
                book_link = "https://aldebaran.ru" + link_soup.get('href')
                r = requests.get(book_link, allow_redirects=True)
                open(save_name, 'wb').write(r.content)
                print('not here', save_name)

# litlib.net

In [ ]:
for page_index in tqdm(range(500)):
    url = "https://www.litlib.net/rating/page" + str(page_index)
    soup = get_page(url)
    links_soup = soup.find_all('a', class_="book")[:-2]
    for link_soup in links_soup:
        url = "https://www.litlib.net" + link_soup.get("href")
        soup = get_page(url)
        book_name = link_soup.text
        book_name = book_name.replace("/", " ")
        download_links = soup.find_all('a', class_="down")
        for link in download_links:
            book_format = ''.join(link.text.split())
            if book_format == "EPUB" or link.get('href')[-4:] == "epub":
                book_link = "https://www.litlib.net" + link.get("href")
                save_name = SAVE_PATH + book_name +  ".epub"
                if os.path.isfile(save_name):
                    print("here", save_name)
                    continue
                r = requests.get(book_link, allow_redirects=True)
                open(save_name, 'wb').write(r.content)
                print("not here", save_name)